In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import math
import seaborn as sb
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision.transforms import transforms as T
from torch import optim
from tqdm import tqdm
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
data1 = pd.read_csv("podaci_one_hot_enkodiranje_DD.csv")

In [3]:
class FillModel(nn.Module):
    def __init__(self):
        super(FillModel, self).__init__()
                    
        self.fnn = nn.Sequential(
            nn.Linear(23, 32),
            nn.ReLU(),
            nn.Dropout(p = 0.2),
            
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(p = 0.2),

            nn.Linear(16, 1),
            nn.ReLU(),
        )
        
    def forward(self, input):
        return self.fnn(input)

In [4]:
model = FillModel()
model.load_state_dict(torch.load("modelFf"))
model.eval()

FillModel(
  (fnn): Sequential(
    (0): Linear(in_features=23, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=16, out_features=1, bias=True)
    (7): ReLU()
  )
)

In [5]:
data2 = data1.drop(["VV", "H"], axis = 1)

In [6]:
data_ff_nan = [x[1:] for x in data2.values if x[4] != x[4]]

In [7]:
data_ff_nan

[array([21.0, 1013.9, 3, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0], dtype=object),
 array([24.0, 1013.9, 4, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0], dtype=object),
 array([26.0, 1013.9, 4, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0], dtype=object),
 array([28.0, 1013.9, 3, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0], dtype=object),
 array([29.0, 1013.9, 4, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=object),
 array([29.0, 1013.9, 3, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1], dtype=object),
 array([30.0, 1012.9, 2, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0], dtype=object),
 array([30.0, 1012.9, 3, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0], dtype=object),
 array([30.0, 1012.9, 2, nan, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [8]:
data_ff_nan = [[*x[:3], *x[4:]] for x in data_ff_nan]

In [9]:
data_ff_nan = torch.Tensor(data_ff_nan)

In [10]:
with torch.no_grad():
    output = model(data_ff_nan)

In [11]:
output = np.array(output.squeeze(1))

In [12]:
index = 0
for (i, x) in enumerate(data1["Ff"]):
    if(x != x):
        data1["Ff"][i] = output[index]
        index+=1 

<ipython-input-12-f5db8239159c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["Ff"][i] = output[index]


In [13]:
data1.isna().sum()

DateTime                                        0
T                                               0
P                                               0
U                                               0
Ff                                              0
Nh                                              0
H                                           53192
VV                                          41399
sss                                             0
DD_Calm, no wind                                0
DD_Wind blowing from the east                   0
DD_Wind blowing from the east-northeast         0
DD_Wind blowing from the east-southeast         0
DD_Wind blowing from the north                  0
DD_Wind blowing from the north-east             0
DD_Wind blowing from the north-northeast        0
DD_Wind blowing from the north-northwest        0
DD_Wind blowing from the north-west             0
DD_Wind blowing from the south                  0
DD_Wind blowing from the south-east             0


In [14]:
data1.to_csv("podaci_FF_popunjeno.csv", index= False)